In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.cm as cm
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from astropy.io import fits
import astropy.io.fits as pyfits
from astropy.wcs import WCS
from astropy.table import Table
import matplotlib.pyplot as pl
import matplotlib.colors as mc
from scipy.interpolate import NearestNDInterpolator
from scipy.optimize import curve_fit 
import scipy.integrate as integrate
import scipy.special as special
from numpy import linspace, array, logspace, sin, cos, pi, arange, sqrt, arctan2, arccos
from mpl_toolkits.mplot3d import Axes3D

import emcee






from collections import OrderedDict


/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
#Grab the existing table columns we want 
cat = Table.read('../3D_CMZ/LBV_plots_stuff/CMZ_cloud_catalogue_data.tex')


cat_index = cat['index']
cloud_list = cat['cloud_name']

###just need this file for the multi-comp clouds
large_cat = Table.read('../3D_CMZ/LBV_plots_stuff/CMZ_cloud_catalogue_data_dup.csv')
large_cat_index = large_cat['index']
large_cat_v = large_cat['v']

### For the stddevs, we need to find them for both the single 
### and multi-component clouds separately 
### then combine into the table 

#Find single v component clouds
single_v = {x for x in list(cloud_list) if list(cloud_list).count(x) == 1}


sing_index =[]
for i in range(len(cloud_list)):
    if cloud_list[i] in single_v:
        sing_index.append(i)
        

#find indexes with multiple velocities 
dup = {x for x in list(cloud_list) if list(cloud_list).count(x) > 1}

dup_index =[]
for i in range(len(cloud_list)):
    if cloud_list[i] in dup:
        dup_index.append(i)
        
id_v_list = []
for i in range(len(large_cat_index)):
    id_v_list.append('{}_{}'.format(large_cat_index[i], large_cat_v[i]))
    
    
mask_files=[]
sing_masks_path = '/Users/danilipman/Documents/Research/UConn/3D_CMZ/Cloud_masks/' 
dup_masks_path = '/Users/danilipman/Documents/Research/UConn/3D_CMZ/Sub_masks/'
for i in range(len(cat_index)):
    if cat_index[i] in cat_index[sing_index]:
        cloud = cloud_list[i]
        
        if '-' in cloud_list[i]:
            filename = sing_masks_path+cloud_list[i] +'/{}_cutout.fits'.format(cloud_list[i].replace("-", "_"))
        if '+' in cloud_list[i]:
            filename = sing_masks_path+cloud_list[i] +'/{}_cutout.fits'.format(cloud_list[i].replace("+", "_"))
        mask_files.append(filename)
        
    if cat_index[i] in cat_index[dup_index]:
        filename = dup_masks_path+'max/{}_{}_HNCO_cube_max.mask.fits'.format(large_cat_index[i],large_cat_v[i] )
        mask_files.append(filename)

        

    

# 3 pigs are IDs 13 (stone), 14 (sticks), and 16b (straw)

In [50]:
stone_cat_row = cat[np.where(cat_index=='13')]
sticks_cat_row = cat[np.where(cat_index=='14')]



xray_tab = Table( names = ('cloud_index', 'cloud_name', 'xray_NEAR_los_dist_pc', 'xray_FAR_los_dist_pc', 'uncertainty', 'reference', 'comment'))
xray_tab['cloud_index'] = xray_tab['cloud_index'].astype(int)
xray_tab['cloud_name'] = xray_tab['cloud_name'].astype(str)
xray_tab['reference'] = xray_tab['reference'].astype(str)
xray_tab['comment'] = xray_tab['comment'].astype(str)


xray_tab.add_row((stone_cat_row['index'], stone_cat_row['cloud_name'], 20, 45, np.nan, 'Clavel+2013', ' '))
xray_tab.add_row((sticks_cat_row['index'], sticks_cat_row['cloud_name'], 20, 45, np.nan, 'Clavel+2013', ' '))

xray_tab.add_row((sticks_cat_row['index'], stone_cat_row['cloud_name'], np.nan, 25, np.nan, 'Marin+2023', ' '))



xray_tab.write("/Users/danilipman/Documents/Research/UConn/CMZ_SYNTH/xray_methods.tex", overwrite=True)




In [48]:
xray_tab

cloud_index,cloud_name,xray_NEAR_los_dist_pc,xray_FAR_los_dist_pc,uncertainty,reference,comment
int64,str32,float64,float64,float64,str32,str32
13,G0.068-0.076,20.0,45.0,nan,Clavel+2013,
14,G0.105-0.08,20.0,45.0,nan,Clavel+2013,
14,G0.068-0.076,nan,25.0,nan,Marin+2023,


In [32]:
stone_cat_row['index'][0]

'13'